In [68]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
import torch_geometric.nn as pyg
from torch_geometric.data import Data
import torch_geometric.transforms as T
from torch_geometric.nn import Node2Vec
import os

load dataset

In [69]:
def get_data(folder="/Users/mac/Desktop/Coding/GraphCNN/cora" , data_name = "cora"):
    data = Planetoid(root=folder ,name=data_name,  
              pre_transform=T.KNNGraph(k=6) ,transform=T.NormalizeFeatures()  )
    return data

In [70]:
data = get_data()

create GCNN model

In [73]:
class MyGCNN(nn.Module):
    
    def __init__(self):
        super(MyGCNN,self).__init__()
        
        self.conv1 = pyg.GCNConv(data.num_features,300 ,cached=True)
        #self.mlp = nn.Linear(300, 300)
        self.conv2 = pyg.GCNConv(300, data.num_classes,cached=True)
        self.reg_params = self.conv1.parameters()
        self.non_reg_params = self.conv2.parameters()
        #self.mlp = nn.Linear(300, data.num_classes)
        

    def forward(self,data):
        x = data.x
        edge_index = data.edge_index
        edge_weight = data.edge_attr
        hid = self.conv1(x=x, edge_index=edge_index,edge_weight = data.edge_attr)
        hid = F.relu(hid)
        hid = F.dropout(hid, training=self.training)
        #hid = self.mlp(hid)
        hid = self.conv2(x=hid ,edge_index=edge_index,edge_weight = data.edge_attr)
        #hid = F.relu(hid)
        out = F.log_softmax(hid,dim=1)
        return out
    

In [76]:
def main():
    
    os.environ["CUDA_VISIBLE_DEVICES"]="0"
    cora = get_data()
    

    NET = MyGCNN()
    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    NET = NET.to(device)
    cora = cora[0].to(device)
    
    #optimizer = torch.optim.Adam(NET.parameters(), lr=1e-3)   
    #optimizer = torch.optim.Adam([dict(params=NET.reg_params, weight_decay=5e-4),
                                  #dict(params=NET.non_reg_params, weight_decay=5e-4)], lr=1e-3)
    optimizer = torch.optim.Adagrad([
        dict(params=NET.reg_params, weight_decay=5e-4),
        dict(params=NET.non_reg_params, weight_decay=5e-4)], 
        lr_decay=0.01, weight_decay=0.01, initial_accumulator_value=0)


    NET.train()
    for epoch in range(500):
        optimizer.zero_grad()
        output = NET(cora)
        loss = F.nll_loss(output[cora.train_mask] , cora.y[cora.train_mask])
        loss.backward()
        optimizer.step()
        print("epoch:",epoch+1 ,"loss:",loss.item())
    
    NET.eval()
    _ , prediction = NET(cora).max(dim=1)
    target = cora.y
    test_correct = prediction[cora.test_mask].eq(target[cora.test_mask]).sum().item()
    test_number = cora.test_mask.sum().item()
    
    print("Accuracy of Test Samples:" , test_correct / test_number)

In [77]:
main()

epoch: 1 loss: 1.9459202289581299
epoch: 2 loss: 1.9224953651428223
epoch: 3 loss: 1.8948427438735962
epoch: 4 loss: 1.864618182182312
epoch: 5 loss: 1.831382155418396
epoch: 6 loss: 1.7977546453475952
epoch: 7 loss: 1.7686859369277954
epoch: 8 loss: 1.7363632917404175
epoch: 9 loss: 1.7024465799331665
epoch: 10 loss: 1.6704761981964111
epoch: 11 loss: 1.6396540403366089
epoch: 12 loss: 1.6129411458969116
epoch: 13 loss: 1.5808777809143066
epoch: 14 loss: 1.5539730787277222
epoch: 15 loss: 1.5252199172973633
epoch: 16 loss: 1.4982720613479614
epoch: 17 loss: 1.4743413925170898
epoch: 18 loss: 1.4519122838974
epoch: 19 loss: 1.4266326427459717
epoch: 20 loss: 1.3988780975341797
epoch: 21 loss: 1.3768144845962524
epoch: 22 loss: 1.3569560050964355
epoch: 23 loss: 1.327013611793518
epoch: 24 loss: 1.3127880096435547
epoch: 25 loss: 1.2881333827972412
epoch: 26 loss: 1.2694188356399536
epoch: 27 loss: 1.2590000629425049
epoch: 28 loss: 1.2274465560913086
epoch: 29 loss: 1.217475175857544
e

epoch: 233 loss: 0.5311967134475708
epoch: 234 loss: 0.5252360701560974
epoch: 235 loss: 0.5280539393424988
epoch: 236 loss: 0.5325965881347656
epoch: 237 loss: 0.5341835021972656
epoch: 238 loss: 0.5312906503677368
epoch: 239 loss: 0.5261034369468689
epoch: 240 loss: 0.530067503452301
epoch: 241 loss: 0.5187565088272095
epoch: 242 loss: 0.5277328491210938
epoch: 243 loss: 0.5263313055038452
epoch: 244 loss: 0.5240254402160645
epoch: 245 loss: 0.5315849184989929
epoch: 246 loss: 0.5195696353912354
epoch: 247 loss: 0.5247431993484497
epoch: 248 loss: 0.5203742980957031
epoch: 249 loss: 0.5271813273429871
epoch: 250 loss: 0.5263353586196899
epoch: 251 loss: 0.5187357068061829
epoch: 252 loss: 0.523711621761322
epoch: 253 loss: 0.5206085443496704
epoch: 254 loss: 0.5206627249717712
epoch: 255 loss: 0.5170743465423584
epoch: 256 loss: 0.5219786763191223
epoch: 257 loss: 0.5189081430435181
epoch: 258 loss: 0.5114572048187256
epoch: 259 loss: 0.509066641330719
epoch: 260 loss: 0.520994722843

epoch: 461 loss: 0.44929084181785583
epoch: 462 loss: 0.44436562061309814
epoch: 463 loss: 0.4389514625072479
epoch: 464 loss: 0.44688573479652405
epoch: 465 loss: 0.4477388560771942
epoch: 466 loss: 0.4560379981994629
epoch: 467 loss: 0.44700273871421814
epoch: 468 loss: 0.45966359972953796
epoch: 469 loss: 0.44719260931015015
epoch: 470 loss: 0.43772509694099426
epoch: 471 loss: 0.4478265643119812
epoch: 472 loss: 0.45769059658050537
epoch: 473 loss: 0.4400230050086975
epoch: 474 loss: 0.45191189646720886
epoch: 475 loss: 0.4452357292175293
epoch: 476 loss: 0.44270187616348267
epoch: 477 loss: 0.440834105014801
epoch: 478 loss: 0.46138882637023926
epoch: 479 loss: 0.43824514746665955
epoch: 480 loss: 0.43535518646240234
epoch: 481 loss: 0.4422002136707306
epoch: 482 loss: 0.44310492277145386
epoch: 483 loss: 0.4489981234073639
epoch: 484 loss: 0.4501461982727051
epoch: 485 loss: 0.45192384719848633
epoch: 486 loss: 0.44399434328079224
epoch: 487 loss: 0.45210298895835876
epoch: 488 l